In [2]:
import os
import PyPDF2
import openai
from dotenv import load_dotenv

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



# 현재 작업 디렉토리 출력
print(f"현재 작업 디렉토리: {os.getcwd()}")



현재 작업 디렉토리: c:\Users\dearh\gafl_chat


In [3]:

# .env 파일 경로를 명시적으로 지정 (절대 경로를 사용)
load_result = load_dotenv(dotenv_path=r"c:\Users\dearh\gafl_chat.env")

# .env 파일이 제대로 로드되었는지 확인
print(f".env 파일 로드 결과: {load_result}")


.env 파일 로드 결과: True


In [4]:


# 환경 변수 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_project = os.getenv("LANGCHAIN_PROJECT")

# 값 출력(테스트 용도)
print(f"OpenAI API Key: {openai_api_key}")
print(f"LangChain API Key: {langchain_api_key}")
print(f"LangChain Tracing: {langchain_tracing}")
print(f"LangChain Project: {langchain_project}")


OpenAI API Key: sk-BCDOda_1zuRAMI6AgI6yue-g9vMx-n8fYJxW4bvWfxT3BlbkFJr2EW6-fvuroktE_h54iKItyShnQzO9Thosf67h3_kA
LangChain API Key: sk-BCDOda_1zuRAMI6AgI6yue-g9vMx-n8fYJxW4bvWfxT3BlbkFJr2EW6-fvuroktE_h54iKItyShnQzO9Thosf67h3_kA
LangChain Tracing: true
LangChain Project: LangChain gafl_chat


In [5]:
# 단계 1: 문서 로드(Load Documents)
loader1 = PyMuPDFLoader("test.pdf")
loader2 = PyMuPDFLoader("학교생활인권규정.pdf")
loader3 = PyMuPDFLoader("아스그수강비규정.pdf")
loader4 = PyMuPDFLoader("외출외박규정.pdf")
loader5 = PyMuPDFLoader("자기주도학습규정.pdf")

docs1 = loader1.load()
docs2 = loader2.load()
docs3 = loader3.load()
docs4 = loader4.load()
docs5 = loader5.load()
all_docs = docs1+docs2+docs3+docs4+docs4+docs5

print(f"문서의 페이지수: {len(all_docs)}")

문서의 페이지수: 201


In [6]:
print(all_docs[10].page_content)

민감정보및고유식별정보처리제한
민감정보및고유식별정보의처리제한민감정보및고유식별정보는원칙적으로처리금지
(
)
개인정보보호법제
조민감정보의처리제한
23
(
)
개인정보처리자는사상ㆍ신념노동조합ㆍ정당의가입ㆍ탈퇴
,
,
①
정치적견해건강성생활등에관한정보그밖에정보주체의사생활을현저히침해할우려가있는개인
,
,
,
정보로서대통령령으로정하는정보이하
민감정보라한다를처리하여서는아니된다다만다음각호의
(
“
”
)
.
,
어느하나에해당하는경우에는그러하지아니하다
개정
. <
2016.3.29.>
개인정보보호법시행령제
조민감정보의범위
18
(
) 법제
조제항각호외의부분본문에서
대통령령으로
23
1
“
정하는정보란다음각호의어느하나에해당하는정보를말한다
다만
공공기관이법제
조제항
”
.
,
18
2
제호부터제호까지의규정에따라다음각호의어느하나에해당하는정보를처리하는경우의해당
5
9
정보는제외한다.
개정
<
2016. 9. 29., 2020. 8. 4.>
유전자검사등의결과로얻어진유전정보
1.
형의실효등에관한법률
제조제호에따른범죄경력자료에해당하는정보
2.
2
5
「
」
3. 개인의신체적생리적행동적특징에관한정보로서특정개인을알아볼목적으로일정한기술적수단을
,
,
통해생성한정보
인종이나민족에관한정보
4.
개인정보보호법제
조고유식별정보의처리제한
24
(
)
개인정보처리자는다음각호의경우를제외하고는
①
법령에따라개인을고유하게구별하기위하여부여된식별정보로서대통령령으로정하는정보이하
고유
(
"
식별정보라한다를처리할수없다
"
)
.
고유식별정보의범위
주민등록번호여권번호운전면허의면허번호외국인등록번호
:
,
,
,
※
개인정보보호법시행령제
조고유식별정보의범위
(
19
(
))
민감정보는다른개인정보의처리에대한동의와별도로동의를받거나법령에서처리를요구하거나
-
허용하는경우에만처리가능
고유식별정보는다른개인정보의처리에대한동의와별도로동의를받거나법령에서구체적으로
-
고유식별정보의처리를요구하거나허용하는경우에는처리가능
다만고유식별정보중주민등록번호는
-
,
법률ㆍ대통령령ㆍ국회규칙ㆍ대법원규칙ㆍ헌법재판소규칙ㆍ중앙선거관리위원회규칙및감사원규칙에서
①
구

In [7]:
all_docs[10].__dict__

{'id': None,
 'metadata': {'source': 'test.pdf',
  'file_path': 'test.pdf',
  'page': 10,
  'total_pages': 147,
  'format': 'PDF 1.5',
  'title': '개인정보보호 업무매뉴얼 개정(2023.2.13).hwp',
  'author': 'co',
  'subject': '',
  'keywords': '',
  'creator': '개인정보보호 업무매뉴얼 개정(2023.2.13).hwp',
  'producer': 'NesPDF',
  'creationDate': "D:20230214095545+09'00'",
  'modDate': "D:20230214095545+09'00'",
  'trapped': ''},
 'page_content': '민감정보및고유식별정보처리제한\n민감정보및고유식별정보의처리제한민감정보및고유식별정보는원칙적으로처리금지\n(\n)\n개인정보보호법제\n조민감정보의처리제한\n23\n(\n)\n개인정보처리자는사상ㆍ신념노동조합ㆍ정당의가입ㆍ탈퇴\n,\n,\n①\n정치적견해건강성생활등에관한정보그밖에정보주체의사생활을현저히침해할우려가있는개인\n,\n,\n,\n정보로서대통령령으로정하는정보이하\n민감정보라한다를처리하여서는아니된다다만다음각호의\n(\n“\n”\n)\n.\n,\n어느하나에해당하는경우에는그러하지아니하다\n개정\n. <\n2016.3.29.>\n개인정보보호법시행령제\n조민감정보의범위\n18\n(\n) 법제\n조제항각호외의부분본문에서\n대통령령으로\n23\n1\n“\n정하는정보란다음각호의어느하나에해당하는정보를말한다\n다만\n공공기관이법제\n조제항\n”\n.\n,\n18\n2\n제호부터제호까지의규정에따라다음각호의어느하나에해당하는정보를처리하는경우의해당\n5\n9\n정보는제외한다.\n개정\n<\n2016. 9. 29., 2020. 8. 4.>\n유전자검사등의결과로얻어진유전정보\n1.\n형의실효등에관한법률\n제조제호에따른범죄경력자료에해당하는정보\n2.

In [8]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(all_docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 505


In [9]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [10]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [11]:
for doc in vectorstore.similarity_search("영상정보처리기기"):
    print(doc.page_content)

,
,
ㆍ영상정보처리기기운영자의영상정보확인방법및장소
ㆍ정보주체의영상정보열람등요구에대한조치
ㆍ영상정보보호를위한기술적관리적및물리적조치방법
,
ㆍ그밖에학교장이영상정보처리기기설치운영및관리에필요한사항
·
영상정보처리기기에대한안전성확보조치
영상정보처리기기관리책임자지정
∙
개인영상정보에대한접근통제및접근권한의제한조치
∙
∙개인영상정보를안전하게저장ㆍ전송할수있는기술의적용
네트워크카메라의경우안전한전송을
(
위한암호화조치개인영상정보파일저장시비밀번호설정등
,
)
∙처리기록의보관및위조ㆍ변조방지를위한조치
개인영상정보의생성일시및열람할경우에열람
(
목적ㆍ열람자ㆍ열람일시등기록ㆍ관리조치등)
∙영상정보처리기기의안전한물리적보관을위한보관시설마련또는잠금장치설치
영상정보처리기기설치시의견수렴및주요출입구안내판설치
참고
뺡
영상정보처리기기운영ㆍ관리방침기안문예시
영상정보처리기기운영ㆍ관리방침예시
영상정보처리기기설치ㆍ운영안내판내용
개인영상정보열람존재확인청구서
·
개인영상정보관리대장
경기도교육청 교육기관 개인정보 보호업무 매뉴얼
100
16
영상정보처리기기 운영ㆍ관리 방침 수립
관련근거
▸개인정보보호법제
조영상정보치리기기의설치ㆍ운영제한
25
(
)
▸개인정보보호법시행령제
조제
조영상정보처리기기운영ㆍ관리방침
22
~
25
(
)
▸교육부개인정보보호지침제장영상정보처리기기설치및운영제
조내지제
조
4
(
66
78
)
업무시기
▸영상정보처리기기도입시운영ㆍ관리방침내용변동시
,
추진사항
영상정보처리기기운영ㆍ관리방침수립ㆍ공개홈페이지등
(
)
∙영상정보처리기기운영ㆍ관리방침은다음사항을포함하여수립하고이를해당기관의학교홈페이지에
게재하여정보주체에게공개법제
조제항시행령제
조
(
25
7
,
25
)
ㆍ영상정보처리기기설치근거및목적
ㆍ영상정보처리기기설치대수설치위치및촬영범위
,
ㆍ관리책임자담당부서및영상정보에대하여접근권한이있는사람
,
ㆍ영상정보의촬영시간보관기관보관장소및처리방법
,
,
ㆍ영상정보처리기기운영자의영상정보확인방법및장소
ㆍ정보주체의영상정보열람등요구에대한조치
조공공기관의영상정보처리기기설치운영사무의위탁에따라다음의내용이포함된문서로

In [12]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [13]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("수업 때 촬영한 내용을 유튜브에 올려도 되나요?")

[Document(metadata={'source': 'test.pdf', 'file_path': 'test.pdf', 'page': 127, 'total_pages': 147, 'format': 'PDF 1.5', 'title': '개인정보보호 업무매뉴얼 개정(2023.2.13).hwp', 'author': 'co', 'subject': '', 'keywords': '', 'creator': '개인정보보호 업무매뉴얼 개정(2023.2.13).hwp', 'producer': 'NesPDF', 'creationDate': "D:20230214095545+09'00'", 'modDate': "D:20230214095545+09'00'", 'trapped': ''}, page_content='수집은가능한가요\n(\n)\n?\n관계법령\n개인정보보호법\n제\n조\n초중등교육법\n제\n조\n15\n,\n·\n25\n｢\n｣\n｢\n｣\n교사가수업시간동영상촬영을한경우이는초중등교육법제\n조제항에따라학생지도에활용할수\n·\n25\n1\n있는행동특성등의자료를작성관리하기위한것으로볼수있는경우에는개인정보보호법제\n조\n·\n15\n제항에서법률에특별한규정이있거나법령상의무를준수하기위하여불가피한경우에해당합니다\n1\n‘\n’\n.\n그러나수업시간에동영상촬영을하더라도촬영된동영상은학생지도등학교생활기록의작성목적범위\n내에서이용되어야하므로정보주체의별도동의없이그목적범위를넘어해당동영상을이용하거나\n제공하는행위는불가능합니다.'),
 Document(metadata={'source': 'test.pdf', 'file_path': 'test.pdf', 'page': 119, 'total_pages': 147, 'format': 'PDF 1.5', 'title': '개인정보보호 업무매뉴얼 개정(2023.2.13).hwp', 'author': 'co', 'subject': '', 'keywords': '', 'creator': '개인정보보호 업무매뉴얼 개정(2023.2.13).hwp', 'producer'

In [14]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context} 

#Question: 
{question} 

#Answer:"""
)

In [15]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch 

In [16]:
chain = prompt | llm | StrOutputParser()

In [17]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "수업을 촬영하고 유튜브로 업로드 해도 되나요?"
response = chain.invoke(question)
print(response)

교사가 수업을 촬영하여 유튜브에 공개할 경우, 학생들에게 개인정보 수집 및 이용에 대한 동의를 받아야 합니다. 특히, 14세 미만의 학생인 경우 법정대리인의 사전 동의도 필요합니다. 또한, 완성된 영상을 유튜브에 활용할 때는 학생 본인 및 보호자의 최종 동의가 필요합니다. 따라서 사전 동의 시 목적 등 활용에 대한 내용을 충분히 이해하기 쉬운 방법으로 알린 후에 동의를 받는 것이 바람직합니다.


In [19]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "개인정보 보호법 15조가 뭐야?"
response = chain.invoke(question)
print(response)

개인정보 보호법 15조는 정보주체의 동의를 받은 경우, 법률에 특별한 규정이 있거나 법령상의 의무 준수를 위해 필요한 경우 등에만 개인정보를 수집할 수 있다는 내용을 담고 있습니다.


In [20]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "개인정보를 비공개로 처리하지 않으면 어떻게 돼?"
response = chain.invoke(question)
print(response)

모르겠습니다.


In [21]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "학부모 주민등록번호를 수집하는 것은 괜찮아?"
response = chain.invoke(question)
print(response)

학부모의 주민등록번호를 수집하는 것은 원칙적으로 불법입니다. 다만, 교육비 납입 소득공제 자료를 국세청에 제출할 때와 같이 법령에서 요구하거나 허용하는 경우에만 예외적으로 수집이 가능합니다.


In [22]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "학부모 주민등록번호를 수집해도 돼?"
response = chain.invoke(question)
print(response)

개인정보보호법 제24조 제2항 제1호에 따라 정보주체의 동의 없이 주민등록번호를 처리할 수 있는 경우가 있습니다. 그러나 일반적으로 학부모의 주민등록번호를 수집하려면 정보주체의 동의를 받아야 합니다. 따라서 학부모의 주민등록번호를 수집하려면 반드시 동의를 받아야 합니다.


In [23]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "학부모 주민등록번호를 조사해도 돼?"
response = chain.invoke(question)
print(response)

모르겠어요. 제공된 문서에서는 학부모 주민등록번호를 조사해도 되는지에 대한 명확한 정보가 없습니다.


In [24]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "CCTV 관련 법"
response = chain.invoke(question)
print(response)

CCTV 관련 법에 따르면, 다음과 같은 규정이 있습니다:

1. 직무상 알게 된 비밀을 누설하거나 직무상 목적 외 사용한 자는 5년 이하 징역 또는 3천만 원 이하 벌금에 처해질 수 있습니다.
2. 안전성 확보에 필요한 보호조치를 취하지 않아 개인정보를 도난, 유출, 변조 또는 훼손당하거나 분실한 자는 2년 이하 징역 또는 2천만 원 이하 벌금에 처해질 수 있습니다.
3. 개인정보를 분리해서 저장, 관리하지 않은 자는 천만 원 이하 과태료에 처해질 수 있습니다.
4. 개인정보의 정정, 삭제 요청에 대한 필요한 조치를 취하지 않고 개인정보를 계속 이용하거나 제3자에게 제공한 자는 3년 이하 징역 또는 2천만 원 이하 벌금에 처해질 수 있습니다.
5. 개인정보의 처리정지 요구에 따라 처리를 중단하지 않고 계속 이용하거나 제3자에게 제공한 자는 3년 이하 징역 또는 2천만 원 이하 벌금에 처해질 수 있습니다.

또한, 유치원 교실 내에 설치된 CCTV의 경우, 다른 학생들이 포함된 영상정보를 열람 제공하기 위해서는 다른 학생들의 법정대리인에게 열람 동의서를 받아야 하며, 다른 학생들이 식별되지 않도록 처리해야 합니다. 

이 외에도, 영상정보를 경찰 등에게 제공할 경우, 개인정보보호법 제18조 제2항 제7호에 따라 공공기관인 경우 범죄 수사 및 공소 제기를 위해 제공할 수 있으며, 공공기관이 아닌 경우에는 적법 절차를 거친 경우에만 제공이 가능합니다.


In [25]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "OG 항목"
response = chain.invoke(question)
print(response)

OG 항목은 다음과 같습니다:

- 학습태도 및 준비불량
- 행사 무단 불참
- 남의 글귀를 가져다가 자기 것으로 발표함 (표절)
- 자습실 규정 위반
- 자기주도학습 불참
- 자기주도학습 시 학습과 관련 없는 행위 적발 시 (잡지, 만화 보기 등)
- 교사에 대한 불손한 언행
- 생활교육위원회 회부
- 지각 (학교 지각, 자기주도학습 지각)
- 외부 음식물 배달 반입 및 취사 행위
- 매점 이용 후 정리 불량 (생일파티, 동아리 모임)
- 타인의 물건을 훼손한 및 허락 없이 사용한 경우


In [28]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "반장이 학생생활교육위원회 회부되면?"
response = chain.invoke(question)
print(response)

반장이 학생생활교육위원회에 회부되면, 학생생활교육위원회는 해당 학생의 신상 및 사안에 관한 제반 의견을 진술할 기회를 주어야 하며, 학생 및 학부모에게도 의견 진술 기회를 제공해야 합니다. 또한, 학교장은 학생생활교육위원회에서 심의된 처분 내용에 대해 재심을 명할 수 있습니다.


In [27]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "조부모가 사망하면 인정결석 일수"
response = chain.invoke(question)
print(response)

조부모가 사망하면 인정결석 일수는 5일입니다.
